In [1]:
# 3.1 DATA SIMULATION (3 puntos) — Julia
# ------------------------------------------------------------
using Random, Distributions, DataFrames, HypothesisTests, Statistics

# (2 pts) Simular n=1000 con X1..X4, D~Bernoulli(0.5), eps~N(0,1)
Random.seed!(123)
n = 1000

X1 = rand(Normal(0,1), n)                # continuo
X2 = rand(Normal(2,1), n)                # continuo
X3 = rand(Bernoulli(0.4), n)             # binario
X4 = rand(Uniform(-1,1), n)              # continuo

D  = rand(Bernoulli(0.5), n)             # tratamiento
ϵ  = rand(Normal(0,1), n)

Y = 2 .* D .+ 0.5 .* X1 .- 0.3 .* X2 .+ 0.2 .* X3 .+ ϵ

df = DataFrame(Y=Y, D=D, X1=X1, X2=X2, X3=X3, X4=X4)

# ------------------------------------------------------------
# (1 pt) Balance check: medias por grupo y t-tests Welch
covs = [:X1, :X2, :X3, :X4]

println("== Medias por grupo ==")
group_means = combine(groupby(df, :D), covs .=> mean)
show(group_means, allrows=true, allcols=true)

println("\n\n== Balance por t-test (Welch) ==")
results = DataFrame(
    Covariable = String[],
    Media_Tratado = Float64[],
    Media_Control = Float64[],
    Diferencia = Float64[],
    t_stat = Float64[],
    p_valor = Float64[]
)

for v in covs
    a = df[df.D .== 1, v]
    b = df[df.D .== 0, v]
    test = UnequalVarianceTTest(a, b)
    push!(results, (
        string(v),
        mean(a),
        mean(b),
        mean(a) - mean(b),
        Statistics.mean(test.t),   # valor t
        pvalue(test)               # valor p
    ))
end

show(results, allrows=true, allcols=true)


== Medias por grupo ==
2×5 DataFrame
 Row │ D      X1_mean     X2_mean  X3_mean   X4_mean    
     │ Bool   Float64     Float64  Float64   Float64    
─────┼──────────────────────────────────────────────────
   1 │ false  -0.0638543  1.99191  0.369072  0.00625674
   2 │  true  -0.0370065  2.04436  0.35534   0.018983

== Balance por t-test (Welch) ==
4×6 DataFrame
 Row │ Covariable  Media_Tratado  Media_Control  Diferencia  t_stat     p_valor  
     │ String      Float64        Float64        Float64     Float64    Float64  
─────┼───────────────────────────────────────────────────────────────────────────
   1 │ X1             -0.0370065    -0.0638543    0.0268478   0.42936   0.667755
   2 │ X2              2.04436       1.99191      0.0524445   0.830258  0.406592
   3 │ X3              0.35534       0.369072    -0.0137324  -0.451083  0.652028
   4 │ X4              0.018983      0.00625674   0.0127263   0.344732  0.730368

In [3]:
# 3.2 ESTIMATING THE AVERAGE TREATMENT EFFECT (3 puntos) — Julia

using DataFrames, GLM, StatsModels, Statistics, Distributions

# (1 pt) ATE simple: Y ~ D
m_simple = lm(@formula(Y ~ D), df)
βs = coef(m_simple); se_s = stderror(m_simple)
ate_simple = βs[2]; se_simple = se_s[2]
t_simple = ate_simple / se_simple
p_simple = 2 * (1 - cdf(TDist(dof_residual(m_simple)), abs(t_simple)))
ci_simple = ate_simple .+ [-1, 1] .* 1.96 .* se_simple

println("== 3.2.1 ATE simple: Y ~ D ==")
println("ATE: $(round(ate_simple, digits=4))  SE: $(round(se_simple, digits=4))  ",
        "95% CI: [$(round(ci_simple[1], digits=4)), $(round(ci_simple[2], digits=4))]  p=$(round(p_simple, sigdigits=4))")

# (1 pt) ATE con controles: Y ~ D + X1 + X2 + X3 + X4
m_ctrl = lm(@formula(Y ~ D + X1 + X2 + X3 + X4), df)
βc = coef(m_ctrl); se_c = stderror(m_ctrl)
ate_ctrl = βc[2]; se_ctrl = se_c[2]
t_ctrl = ate_ctrl / se_ctrl
p_ctrl = 2 * (1 - cdf(TDist(dof_residual(m_ctrl)), abs(t_ctrl)))
ci_ctrl = ate_ctrl .+ [-1, 1] .* 1.96 .* se_ctrl

println("\n== 3.2.2 ATE con controles ==")
println("ATE: $(round(ate_ctrl, digits=4))  SE: $(round(se_ctrl, digits=4))  ",
        "95% CI: [$(round(ci_ctrl[1], digits=4)), $(round(ci_ctrl[2], digits=4))]  p=$(round(p_ctrl, sigdigits=4))")

# (1 pt) Comparación
delta_ate = ate_ctrl - ate_simple
delta_se  = se_ctrl - se_simple
ratio_se  = se_ctrl / se_simple

println("\n== 3.2.3 Comparación ==")
println("Cambio en ATE (controles - simple): $(round(delta_ate, digits=4))")
println("Cambio en SE: $(round(delta_se, digits=4))   Ratio SE (ctrl/simple): $(round(ratio_se, digits=3))")




== 3.2.1 ATE simple: Y ~ D ==
ATE: 1.9732  SE: 0.072  95% CI: [1.8321, 2.1142]  p=0.0

== 3.2.2 ATE con controles ==
ATE: 1.9787  SE: 0.0626  95% CI: [1.8559, 2.1014]  p=0.0

== 3.2.3 Comparación ==
Cambio en ATE (controles - simple): 0.0055
Cambio en SE: -0.0093   Ratio SE (ctrl/simple): 0.87
Nota: En un RCT balanceado, el ATE cambia poco; agregar controles predictivos reduce varianza.


In [33]:
import Pkg; Pkg.add("Convex")

   Resolving package versions...
   Installed CodecBzip2 ─────────── v0.8.5
   Installed CommonSubexpressions ─ v0.3.1
   Installed BenchmarkTools ─────── v1.6.0
   Installed StructTypes ────────── v1.11.0
   Installed MutableArithmetics ─── v1.6.4
   Installed AMD ────────────────── v0.5.3
   Installed Convex ─────────────── v0.16.4
   Installed DiffRules ──────────── v1.15.1
   Installed DiffResults ────────── v1.1.0
   Installed MathOptInterface ───── v1.44.0
   Installed JSON3 ──────────────── v1.14.3
   Installed AbstractTrees ──────── v0.4.5
   Installed ForwardDiff ────────── v1.2.1
   Installed LDLFactorizations ──── v0.10.1
    Updating `C:\Users\User\.julia\environments\v1.11\Project.toml`
  [f65535da] + Convex v0.16.4
    Updating `C:\Users\User\.julia\environments\v1.11\Manifest.toml`
  [14f7f29c] + AMD v0.5.3
  [1520ce14] + AbstractTrees v0.4.5
  [6e4b80f9] + BenchmarkTools v1.6.0
  [523fee87] + CodecBzip2 v0.8.5
  [bbf7d656] + CommonSubexpressions v0.3.1
  [f65535da] + Co

In [39]:
import Pkg; Pkg.add("SCS")

   Resolving package versions...
   Installed SCS ──────────── v2.2.0
   Installed SCS_jll ──────── v3.2.8+0
   Installed OpenBLAS32_jll ─ v0.3.29+0
    Updating `C:\Users\User\.julia\environments\v1.11\Project.toml`
  [c946c3f1] + SCS v2.2.0
    Updating `C:\Users\User\.julia\environments\v1.11\Manifest.toml`
  [c946c3f1] + SCS v2.2.0
  [656ef2d0] + OpenBLAS32_jll v0.3.29+0
  [f4f2fc5b] + SCS_jll v3.2.8+0
Precompiling project...
    426.7 ms  ✓ OpenBLAS32_jll
    515.4 ms  ✓ SCS_jll
  33496.0 ms  ✓ SCS
  3 dependencies successfully precompiled in 37 seconds. 339 already precompiled.


In [45]:
import Pkg; Pkg.add("MathOptInterface")

   Resolving package versions...
    Updating `C:\Users\User\.julia\environments\v1.11\Project.toml`
  [b8f27783] + MathOptInterface v1.44.0
  No Changes to `C:\Users\User\.julia\environments\v1.11\Manifest.toml`


In [35]:
"""
Solve LASSO / Elastic Net with Convex.jl + SCS.

Minimizes (1/T) * ||Y - X*b||² + γ * ||b||₁ + λ * ||b||₂²

Notes
-----
- No intercept term inside the solver. If you want an intercept, center Y and
  standardize X before optimization, then recover intercept later in OLS.
- Suitable for variable selection: selected = {j : b̂_j ≠ 0}.

Parameters
----------
Y :: AbstractVector{<:Real}
X :: AbstractMatrix{<:Real}
γ :: Real                  # L1 penalty ≥ 0 (LASSO intensity)
λ :: Real = 0.0            # L2 penalty ≥ 0 (Elastic-Net; λ=0 → pure LASSO)

Returns
-------
b_hat :: Vector{Float64}   # penalized solution
b_ls  :: Vector{Float64}   # unpenalized OLS (reference)
"""
function LassoEN(Y, X, γ, λ::Real=0.0)
    T, K = size(X)
    Y = collect(float.(Y))
    X = collect(float.(X))

    # Unpenalized least squares (reference)
    b_ls = X \ Y

    # Precompute quadratic + linear parts of (1/T)||Y - Xb||² up to constant
    Q = Symmetric((X' * X) / T)      # ensure symmetric PSD
    c = (X' * Y) / T

    # Decision variable and objective
    b = Variable(K)
    obj = quadform(b, Q) - 2dot(c, b) + γ * norm(b, 1)
    if λ > 0
        obj += λ * sumsquares(b)
    end

    problem = minimize(obj)
    solve!(problem, SCS.Optimizer; silent=true)

    b_hat = if problem.status == MOI.OPTIMAL || problem.status == MOI.ALMOST_OPTIMAL
        vec(evaluate(b))
    else
        fill(NaN, K)
    end
    return b_hat, b_ls
end






LassoEN

In [47]:
using LinearAlgebra        # Symmetric, dot, etc.
using Convex               # Variable, minimize, quadform, norm, sumsquares
using SCS                  # SCS.Optimizer
using MathOptInterface
const MOI = MathOptInterface



"""
Select covariates X1..X4 via Convex-LASSO with a γ-grid and BIC model choice.

Workflow
--------
1) Build Xc = [X1,X2,X3,X4], y = Y, exclude D from selection.
2) Standardize columns of Xc and center y for stable penalization.
3) For each γ in a log-grid, solve LassoEN(yc, Xs, γ), compute:
       RSS_γ = ||yc - Xs * b̂(γ)||²,
       k_γ   = number of nonzeros in b̂(γ),
       BIC_γ = n*log(RSS_γ/n) + k_γ*log(n).
4) Choose γ* with minimal BIC. Selected vars = {j : b̂_j(γ*) ≠ 0}.

Parameters
----------
df :: DataFrame   # must contain :Y, :D, :X1, :X2, :X3, :X4
gammas :: AbstractVector{<:Real}
    Penalty grid to scan (default: 40 values log-spaced)

Returns
-------
selected :: Vector{Symbol}          # chosen covariates from X1..X4
γstar    :: Float64                 # BIC-optimal gamma
b_star   :: Vector{Float64}         # coefficients at γ*
"""
function select_covariates_convex(df::DataFrame; gammas::AbstractVector{<:Real}=exp.(range(log(1e-3), log(10.0), length=40)))
    # Design matrices (exclude D in selection step)
    Xc = Matrix(select(df, [:X1, :X2, :X3, :X4]))
    y  = Array(df.Y)
    n, p = size(Xc)

    # Standardize X, center y (no intercept in solver)
    μx = vec(mean(Xc, dims=1))
    σx = vec(std(Xc, dims=1, corrected=true)); σx[σx .== 0.0] .= 1.0
    Xs = (Xc .- μx') ./ σx'
    μy = mean(y)
    yc = y .- μy

    # Scan γ-grid, compute BIC
    best_idx = 0
    best_bic = Inf
    best_b   = zeros(p)
    for (i, γ) in enumerate(gammas)
        b̂, _ = LassoEN(yc, Xs, γ, 0.0)        # pure LASSO here
        yhat  = Xs * b̂
        rss   = sum((yc .- yhat).^2)
        k     = count(!iszero, b̂)             # nonzeros, no intercept inside
        bic   = n * log(rss / n + eps()) + k * log(n)
        if bic < best_bic
            best_bic = bic
            best_idx = i
            best_b   = b̂
        end
    end
    γstar = gammas[best_idx]

    # Selected variables (indices where b̂ ≠ 0)
    sel_idx = findall(!iszero, best_b)
    selected = Symbol.(["X1","X2","X3","X4"][sel_idx])

    return selected, γstar, best_b
end

selected_vars, γstar, b_star = select_covariates_convex(df)
println("Selected by Convex-LASSO (BIC, γ* = ", round(γstar, digits=6), "): ", selected_vars)


Selected by Convex-LASSO (BIC, γ* = 0.001): [:X1, :X2, :X3, :X4]


LoadError: MethodError: no method matching fit(::Type{LinearModel}, ::Tuple{FormulaTerm{Term, Term}, Vararg{Term, 4}}, ::DataFrame, ::Nothing)
The function `fit` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  fit(::Type{T}, [91m::FormulaTerm[39m, ::Any, ::Any...; contrasts, kwargs...) where T<:RegressionModel
[0m[90m   @[39m [36mStatsModels[39m [90mC:\Users\User\.julia\packages\StatsModels\YNwJ1\src\[39m[90m[4mstatsmodel.jl:78[24m[39m
[0m  fit([91m::Type{D}[39m, ::Any...) where D<:Distribution
[0m[90m   @[39m [35mDistributions[39m [90mC:\Users\User\.julia\packages\Distributions\YQSrn\src\[39m[90m[4mgenericfit.jl:47[24m[39m
[0m  fit(::Type{T}, [91m::FormulaTerm[39m, ::Any, ::Any...; contrasts, kwargs...) where T<:StatisticalModel
[0m[90m   @[39m [36mStatsModels[39m [90mC:\Users\User\.julia\packages\StatsModels\YNwJ1\src\[39m[90m[4mstatsmodel.jl:78[24m[39m
[0m  ...
